In [2]:
import pandas as pd

# Daten aus CSV laden und nach Datumsbereich filtern
data = pd.read_csv('../daten/Gesamtdatensatz.csv')
data = data[(data['timestamp'] >= '2023-04-01') & (data['timestamp'] <= '2024-03-31')]
data

,timestamp,location_id,location_name,ltr_label,rtl_label,weather_condition,temperature,pedestrians_count,unverified,collection_type,...,zone_3_pedestrians_count,zone_3_ltr_pedestrians_count,zone_3_rtl_pedestrians_count,zone_3_adult_pedestrians_count,zone_3_child_pedestrians_count,zone_99_pedestrians_count,zone_99_ltr_pedestrians_count,zone_99_rtl_pedestrians_count,zone_99_adult_pedestrians_count,zone_99_child_pedestrians_count
52712,2023-04-01T00:00:00Z,329,Bahnhofstrasse (Mitte),Hauptbahnhof,Bürkliplatz,rain,8.0,185,False,measured,...,91.0,67.0,24.0,90.0,1.0,NaN,NaN,NaN,NaN,NaN
52713,2023-04-01T00:00:00Z,331,Bahnhofstrasse (Nord),Bürkliplatz,Hauptbahnhof,rain,8.0,165,False,measured,...,71.0,20.0,51.0,71.0,0.0,NaN,NaN,NaN,NaN,NaN
52714,2023-04-01T00:00:00Z,330,Bahnhofstrasse (Süd),Bürkliplatz,Hauptbahnhof,rain,8.0,70,False,measured,...,36.0,24.0,12.0,32.0,4.0,NaN,NaN,NaN,NaN,NaN
52715,2023-04-01T00:00:00Z,670,Lintheschergasse,Hauptbahnhof,Uraniastraße,rain,8.0,0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52716,2023-04-01T01:00:00Z,329,Bahnhofstrasse (Mitte),Hauptbahnhof,Bürkliplatz,cloudy,8.6,108,False,measured,...,36.0,24.0,12.0,36.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87747,2024-03-30T22:00:00Z,670,Lintheschergasse,Hauptbahnhof,Uraniastraße,rain,9.7,48,False,measured,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87748,2024-03-30T23:00:00Z,329,Bahnhofstrasse (Mitte),Hauptbahnhof,Bürkliplatz,rain,9.0,307,False,measured,...,124.0,58.0,66.0,124.0,0.0,NaN,NaN,NaN,NaN,NaN
87749,2024-03-30T23:00:00Z,331,Bahnhofstrasse (Nord),Bürkliplatz,Hauptbahnhof,rain,9.0,298,False,measured,...,150.0,47.0,103.0,147.0,3.0,NaN,NaN,NaN,NaN,NaN
87750,2024-03-30T23:00:00Z,330,Bahnhofstrasse (Süd),Bürkliplatz,Hauptbahnhof,rain,9.0,180,False,measured,...,70.0,28.0,42.0,65.0,5.0,NaN,NaN,NaN,NaN,NaN


In [3]:
# Pivot Tabelle
pivot = pd.pivot_table(
    data,
    index='location_name',
    values=['pedestrians_count', 'child_pedestrians_count', 'adult_pedestrians_count'],
    aggfunc='sum',
    fill_value=0
).reset_index()
pivot


,location_name,adult_pedestrians_count,child_pedestrians_count,pedestrians_count
0,Bahnhofstrasse (Mitte),13525360,220881,13746241
1,Bahnhofstrasse (Nord),6156714,48271,6204985
2,Bahnhofstrasse (Süd),7824223,264970,8089193
3,Lintheschergasse,182687,1892,184579


In [4]:
# Spalten umbenennen
pivot = pivot.rename(columns={
    'location_name': 'Standorte', 
    'child_pedestrians_count': 'Anzahl Kinder',
    'adult_pedestrians_count': 'Anzahl Erwachsene',
    'pedestrians_count': 'Total',
})
pivot

,Standorte,Anzahl Erwachsene,Anzahl Kinder,Total
0,Bahnhofstrasse (Mitte),13525360,220881,13746241
1,Bahnhofstrasse (Nord),6156714,48271,6204985
2,Bahnhofstrasse (Süd),7824223,264970,8089193
3,Lintheschergasse,182687,1892,184579


In [5]:
# Anteil Kinder berechnen
pivot['Anteil Kinder'] = pivot['Anzahl Kinder'] / pivot['Total']

# NaN-Werte bei möglicher Division durch 0 ersetzen
pivot = pivot.fillna(0)

pivot

,Standorte,Anzahl Erwachsene,Anzahl Kinder,Total,Anteil Kinder
0,Bahnhofstrasse (Mitte),13525360,220881,13746241,0.016068
1,Bahnhofstrasse (Nord),6156714,48271,6204985,0.007779
2,Bahnhofstrasse (Süd),7824223,264970,8089193,0.032756
3,Lintheschergasse,182687,1892,184579,0.010250


In [16]:
import altair as alt

bar_width = 80
bar_spacing = 0.5 * bar_width

chart = alt.Chart(pivot, ).mark_bar(size=bar_width).encode(
    x=alt.X('Standorte').axis(
        labelAngle=-45,
        labelOffset=14,
    ), 
    y=alt.Y('Anteil Kinder:Q').axis(
        format='.1%',
        tickCount=7,
    ),
    tooltip=[
        alt.Tooltip('Anteil Kinder', format='.1%'),
        alt.Tooltip('Anzahl Kinder', format='i'),
        alt.Tooltip('Anzahl Erwachsene', format='i'),
        alt.Tooltip('Total', format='i'),
    ],
).properties(
    width={"step": bar_width + bar_spacing},
    height=400
).configure_axis(
    titleFontSize=14, 
    titleFontWeight="bold",
    labelFontSize=12,
    labelFontWeight="normal",
    labelAngle=0,
)
chart

alt.Chart(...)

In [13]:
import json

# In Dictionary umwandeln
spec = chart.to_dict()

# Dataset umbenennen für einfacheren Zugriff im Frontend
new_name = 'values'
old_name = spec['data']['name']
spec['data']['name'] = new_name
spec['datasets'][new_name] = spec['datasets'].pop(old_name)

# Spec in eine Datei schreiben
with open("fokusfrage.json", "w") as f:
    json.dump(spec, f, indent=2)